## TEST_DATA

In [1]:
import pandas as pd

In [2]:
review_3p = pd.read_excel("./test_data/(전달용) 쿠팡_리뷰 20240223.xlsx", sheet_name="(RAW) 쿠팡_3P")
review_rocket = pd.read_excel("./test_data/(전달용) 쿠팡_리뷰 20240223.xlsx", sheet_name="(RAW) 쿠팡_로켓")

In [3]:
review_3p_comments = review_3p["상품평 코멘트"].dropna().tolist()
review_rocket_comments = review_rocket["리뷰내용"].dropna().tolist()

In [4]:
combined_reviews = review_3p_comments + review_rocket_comments

In [35]:
test_data = combined_reviews[:300]

## Import

In [34]:
# LLM 객체 생성
from langchain.chat_models import ChatOpenAI

from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

## LLM_1 : Aspect Based Sentiment Analysis

In [7]:
# 환경변수
from dotenv import load_dotenv

load_dotenv()

True

In [19]:
# LLM_1
ABSA_llm = ChatOpenAI(
    temperature=0.7,  # 창의성
    model_name="gpt-4o",  # 모델명
)

In [32]:
ABSA_template = """
    Define the sentiment elements as follows:
    
    − The 'aspect term' refers to a specific feature, attribute, or aspect of an item, product, or service that a user might comment on. If implicit, the aspect term can be 'null'.
    − The 'opinion term' reflects the sentiment or attitude toward a specific aspect. It can also be 'null' for implicit opinions.
    − The 'aspect category' is the general category to which the aspect belongs. Examples may include 'quality', 'pricing', 'availability', 'appearance', 'functionality', 'service', 'general', and other suitable categories as applicable across diverse domains.
    − The 'sentiment polarity' expresses whether the sentiment is 'positive', 'negative', or 'neutral'.

    Based on the provided definitions, identify and organize all sentiment elements from the following text by grouping them by sentiment polarity. 
    
    Please answer in Korean.

    Text to analyze:
    {text}
"""

In [45]:
# 프롬프트 생성
ABSA_prompt = ChatPromptTemplate.from_template(ABSA_template)

# 체인 생성 (프롬프트와 LLM 연결)
ABSA_chain = LLMChain(llm=ABSA_llm, prompt=ABSA_prompt)

# 체인을 통해 combined_reviews 텍스트로 실행
ABSA_result = ABSA_chain.run({"text": test_data})

print(ABSA_result)

### 긍정적 감정 요소

1. **측면 용어**: 관절 건강, 섭취, 배송, 캐나다 직수입, 가격, 하루 1정
   - **의견 용어**: 간편, 좋아요, 마음에 들어요, 합리적, 좋습니다
   - **측면 범주**: 품질, 가격, 편리성
   - **감정 극성**: 긍정적

2. **측면 용어**: MSM, 관절보호, 캐나다 수입품, 120알, 4개월분
   - **의견 용어**: 믿고 먹을 수 있네요, 넉넉히 두고두고 먹을 수 있어 좋았습니다
   - **측면 범주**: 품질, 신뢰성
   - **감정 극성**: 긍정적

3. **측면 용어**: 관절, 손목, 무릎, 남편
   - **의견 용어**: 최고예요, 편해요, 괜찮아졌다고해요
   - **측면 범주**: 효과, 건강
   - **감정 극성**: 긍정적

4. **측면 용어**: 관절, 식물성, 계단
   - **의견 용어**: 많이 좋아졌네요
   - **측면 범주**: 효과, 건강
   - **감정 극성**: 긍정적

5. **측면 용어**: 성분, 관절 통증 감소
   - **의견 용어**: 좋을것같더라구요, 기대되요
   - **측면 범주**: 효과, 기대
   - **감정 극성**: 긍정적

6. **측면 용어**: MSM, 관절, 용량
   - **의견 용어**: 좋다고 하시네용, 만족스럽네요
   - **측면 범주**: 효과, 만족도
   - **감정 극성**: 긍정적

7. **측면 용어**: 관절, 연골, 통증 감소
   - **의견 용어**: 효과를 볼수 있다고 하여, 기대합니다
   - **측면 범주**: 효과, 기대
   - **감정 극성**: 긍정적

8. **측면 용어**: 관절, 연골, 통증 감소
   - **의견 용어**: 좋다고 하시네요, 건강하셨으면 좋겠어용
   - **측면 범주**: 효과, 건강
   - **감정 극성**: 긍정적

9. **측면 용어**: 관절, 연골, 통증 감소
   - **의견 용어**: 좋다고 하시네요, 건강하셨으면 좋겠어용
   - **

## Chat LLM

In [46]:
# LLM 객체 생성
from langchain.chat_models import ChatOpenAI

from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [47]:
# LLM_2
chat_llm = ChatOpenAI(
    temperature=0.1,  # 창의성
    model_name="gpt-4o",  # 모델명
)

In [59]:
chat_template = """
    You are an assistant for question-answering tasks.
    Use the following pieces of 'information' to answer the question.

    Information:
    {ABSA_text}

    Question:
    {chat_question}

    Please answer in Korean.
"""

In [60]:
# 프롬프트 생성
chat_prompt = ChatPromptTemplate.from_template(chat_template)

# 체인 생성 (프롬프트와 LLM 연결)
chat_chain = LLMChain(llm=chat_llm, prompt=chat_prompt)

In [62]:
# 유저와의 대화 부분
while True:

    # 유저의 질문 받기
    user_question = input("사용자의 질문을 입력하세요 (종료하려면 'exit' 입력): ")

    if user_question.lower() == 'exit':
        print("대화를 종료합니다.")
        break

    # 체인을 통해 combined_reviews 텍스트로 실행
    chat_result = chat_chain.run({"ABSA_text": ABSA_result, "chat_question": user_question})

    print(chat_result)

긍정적인 측면에서 보면, 관절 건강과 관련된 제품은 품질, 가격, 편리성, 신뢰성, 효과, 건강, 기대, 만족도 등 다양한 측면에서 긍정적인 평가를 받고 있습니다. 특히, 관절 통증 감소와 관련된 효과가 좋다는 의견이 많고, 하루 1정으로 간편하게 섭취할 수 있다는 점이 장점으로 언급되고 있습니다. 또한, 캐나다 직수입 제품이라는 점에서 신뢰성을 더하고 있습니다.

반면, 부정적인 측면에서는 알약의 크기가 크다는 점이 불편하다는 의견이 있습니다. 목에 걸리거나 목구멍이 찢어질 것 같다는 불편함이 제기되고 있습니다.

중립적인 측면에서는 약의 효과에 대해 아직 잘 모르겠다는 의견이 있으며, 효과가 있기를 기대하는 마음이 표현되고 있습니다.

종합적으로 보면, 제품의 효과와 편리성, 신뢰성 측면에서 긍정적인 평가가 많지만, 알약 크기와 관련된 불편함이 일부 존재합니다.
대화를 종료합니다.


## Marketing LLM